In [1]:
# Import dependencies
import pandas as pd
import os
# import csv

## Import The Data

In [2]:
# Specify Filepaths
locations_inpath = "Resources/CitiesWGeolocation.csv"
cost_analysis_inpath = "Resources/cost_analysis.csv"
weather_inpath = "Resources/weather.csv"
yelp_inpath = "Resources/final_activities.csv"
chat_inpath = "Resources/chatgpt_activities.csv"
activity_inpath = "Resources/activity_list.csv"

In [3]:
# Import Files
locations_raw_df = pd.read_csv(locations_inpath,index_col=[0])
cost_analysis_df = pd.read_csv(cost_analysis_inpath,index_col=[0])
weather_df = pd.read_csv(weather_inpath,index_col=[0])
yelp_df = pd.read_csv(yelp_inpath)
chat_df = pd.read_csv(chat_inpath,index_col=[0])
activity_df = pd.read_csv(activity_inpath)

### Clean Datasets Before Merging

In [4]:
locations_raw_df.info()
locations_raw_df["Country"] = locations_raw_df["Country"].str.replace('\u200b',"")
locations_raw_df["Country"] = locations_raw_df["Country"].str.replace('Kingdom of the Netherlands','Netherlands')
locations_raw_df["Country"] = locations_raw_df["Country"].str.replace('Sr Lanka','Sri Lanka')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 220 entries, 0 to 236
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   City            220 non-null    object 
 1   State/Province  75 non-null     object 
 2   Country         220 non-null    object 
 3   Latitude        220 non-null    float64
 4   Longitude       220 non-null    float64
dtypes: float64(2), object(3)
memory usage: 10.3+ KB


In [5]:
#cost_analysis_df.sort_values(by="cost_rank")

In [6]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2640 entries, 0 to 2639
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Latitude               2640 non-null   float64
 1   Longitude              2640 non-null   float64
 2   Month                  2640 non-null   int64  
 3   Average Temperature    2640 non-null   float64
 4   Average No Sunny Days  2640 non-null   float64
dtypes: float64(4), int64(1)
memory usage: 123.8 KB


In [7]:
yelp_df.info()
#change objects to strings and fix string formatting

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   latitude   162 non-null    float64
 1   longitude  162 non-null    float64
 2   act1       162 non-null    object 
 3   act2       147 non-null    object 
 4   act3       144 non-null    object 
 5   act4       142 non-null    object 
 6   act5       141 non-null    object 
dtypes: float64(2), object(5)
memory usage: 9.0+ KB


In [8]:
df_list = [locations_raw_df,cost_analysis_df,weather_df,chat_df,yelp_df]

for i in range(len(df_list)): 
    df_list[i].columns = df_list[i].columns.str.lower()
    for x in range(len(df_list[i].columns)):
        df = df_list[i]
        col = df.columns[x]
        try:
            df[col] = df[col].str.title()
            df[col] = df[col].str.strip()
            df[col] = df[col].astype(str)
        except:
            df[col] = df[col].round()
            pass


cost_analysis_df = cost_analysis_df.rename(columns={"total_rank":"cost_rank"})
cost_analysis_df = cost_analysis_df[["city","state/province","country","latitude","longitude","cost_rank"]]
weather_df = weather_df.rename(columns={"average temperature":"avg_temp","average no sunny days":"avg_num_sun_days"})

## Merge All Dataframes Together

In [9]:
# Merge Locations onto Cost Analysis (step1_df)
step1_df = pd.merge(locations_raw_df,cost_analysis_df,how='inner',on=["city","country","latitude","longitude","state/province"])
step1_df = step1_df.drop_duplicates(subset=["city","country","latitude","longitude"])
step1_df.sort_values(by=["cost_rank"])

,city,state/province,country,latitude,longitude,cost_rank
109,Siem Reap,nan,Cambodia,13.0,104.0,1.0
99,São Paulo,nan,Brazil,-24.0,-47.0,1.0
98,Jakarta,nan,Indonesia,-6.0,107.0,1.0
96,Chiang Mai,nan,Thailand,19.0,99.0,1.0
94,Bucharest,nan,Romania,44.0,26.0,1.0
...,...,...,...,...,...,...
121,Berlin,nan,Germany,53.0,13.0,5.0
126,New York,nan,United States,41.0,-74.0,5.0
171,Brussels,nan,Belgium,51.0,4.0,5.0
201,Rotterdam,nan,Netherlands,52.0,4.0,5.0


In [10]:
# Merge (step1_df) onto Weather (step2_df)  ---REDO, you've dropped the calendar months by only keeping the first time the
#city appears
step2_df = pd.merge(step1_df,weather_df,how='inner',on=["latitude","longitude"])
step2_df = step2_df.drop_duplicates(subset=["city","country","latitude","longitude"],keep='first')
step2_df.sort_values(by="cost_rank")

,city,state/province,country,latitude,longitude,cost_rank,month,avg_temp,avg_num_sun_days
1440,Krakow,nan,Poland,50.0,20.0,1.0,4,6.0,11.0
516,Marrakech,nan,Morocco,32.0,-8.0,1.0,4,20.0,10.0
2172,Pokhara,nan,Nepal,28.0,84.0,1.0,4,23.0,11.0
1584,Colombo,nan,Sri Lanka,7.0,80.0,1.0,4,27.0,7.0
624,Jaipur,Rajasthan,India,27.0,76.0,1.0,4,33.0,12.0
...,...,...,...,...,...,...,...,...,...
960,Odense,Fyn,Denmark,55.0,10.0,5.0,4,7.0,10.0
2328,Munich,nan,Germany,48.0,12.0,5.0,4,8.0,10.0
2304,Stockholm,nan,Sweden,59.0,18.0,5.0,4,5.0,13.0
1680,New York,nan,United States,41.0,-74.0,5.0,4,12.0,10.0


In [11]:
# Merge (step2_df) onto Chat (step3_df)
step3_df = pd.merge(step2_df,chat_df,how='left',on=['city','country'])
step3_df.sort_values(by="cost_rank")

,city,state/province,country,latitude,longitude,cost_rank,month,avg_temp,avg_num_sun_days,act1,act2,act3,act4,act5
104,Krakow,nan,Poland,50.0,20.0,1.0,4,6.0,11.0,NaN,NaN,NaN,NaN,NaN
36,Marrakech,nan,Morocco,32.0,-8.0,1.0,4,20.0,10.0,NaN,NaN,NaN,NaN,NaN
163,Pokhara,nan,Nepal,28.0,84.0,1.0,4,23.0,11.0,Hiking,Tours,Hot Air Balloons,Spas,Horsebackriding
116,Colombo,nan,Sri Lanka,7.0,80.0,1.0,4,27.0,7.0,Beaches,Tours,Spas,Snorkeling,Sailing
44,Jaipur,Rajasthan,India,27.0,76.0,1.0,4,33.0,12.0,Tours,Hot_Air_Balloons,Horsebackriding,Spas,Hiking
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,Odense,Fyn,Denmark,55.0,10.0,5.0,4,7.0,10.0,NaN,NaN,NaN,NaN,NaN
176,Munich,nan,Germany,48.0,12.0,5.0,4,8.0,10.0,NaN,NaN,NaN,NaN,NaN
174,Stockholm,nan,Sweden,59.0,18.0,5.0,4,5.0,13.0,NaN,NaN,NaN,NaN,NaN
124,New York,nan,United States,41.0,-74.0,5.0,4,12.0,10.0,NaN,NaN,NaN,NaN,NaN


In [12]:
locations_raw_df["country"].nunique()

83

In [13]:
# Merge (step3_df) onto Yelp (step4_df)
step4_df=pd.merge(step3_df,yelp_df,how='inner',on=['latitude','longitude'])
step4_df.sort_values(by='cost_rank')

,city,state/province,country,latitude,longitude,cost_rank,month,avg_temp,avg_num_sun_days,act1_x,act2_x,act3_x,act4_x,act5_x,act1_y,act2_y,act3_y,act4_y,act5_y
164,La Paz,nan,Bolivia,-16.0,-68.0,1.0,4,8.0,11.0,Hiking,Tours,Hot Springs,Hang Gliding,Horsebackriding,Ziplining,nan,nan,nan,nan
94,Krakow,nan,Poland,50.0,20.0,1.0,4,6.0,11.0,NaN,NaN,NaN,NaN,NaN,Ziplining,Beaches,Hot_Air_Balloons,Hiking,Tours
95,Kuala Lumpur,nan,Malaysia,3.0,102.0,1.0,4,27.0,8.0,NaN,NaN,NaN,NaN,NaN,Ziplining,Hot_Air_Balloons,Hiking,Massage,Tours
96,Ho Chi Minh City,nan,Vietnam,11.0,107.0,1.0,4,29.0,11.0,Tours,Spas,Massage,Hot Springs,Hiking,Ziplining,nan,nan,nan,nan
92,Rio De Janeiro,nan,Brazil,-23.0,-43.0,1.0,4,23.0,6.0,NaN,NaN,NaN,NaN,NaN,Beaches,Ziplining,Tours,Hiking,Massage
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,Brussels,nan,Belgium,51.0,4.0,5.0,4,12.0,10.0,NaN,NaN,NaN,NaN,NaN,Ziplining,Massage,Tours,Hanggliding,Beaches
143,Stockholm,nan,Sweden,59.0,18.0,5.0,4,5.0,13.0,NaN,NaN,NaN,NaN,NaN,Ziplining,Tours,Massage,Beaches,Horsebackriding
142,London,nan,United Kingdom,52.0,-0.0,5.0,4,12.0,9.0,NaN,NaN,NaN,NaN,NaN,Tours,Ziplining,Massage,Horsebackriding,Hot_Air_Balloons
161,Copenhagen,nan,Denmark,56.0,13.0,5.0,4,6.0,10.0,NaN,NaN,NaN,NaN,NaN,Tours,Ziplining,Beaches,Hiking,Massage


In [14]:
step4_df['act1_x'].fillna(step4_df['act1_y'],inplace=True)
step4_df['act2_x'].fillna(step4_df['act2_y'],inplace=True)
step4_df['act3_x'].fillna(step4_df['act3_y'],inplace=True)
step4_df['act4_x'].fillna(step4_df['act4_y'],inplace=True)
step4_df['act5_x'].fillna(step4_df['act5_y'],inplace=True)
step4_df.drop(['act1_y','act2_y','act3_y','act4_y','act5_y'],axis=1,inplace=True)
step4_df.columns=step4_df.columns.str.rstrip('_x')
step4_df.head()

,city,state/province,country,latitude,longitude,cost_rank,month,avg_temp,avg_num_sun_days,act1,act2,act3,act4,act5
0,Lexington,Kentucky,United States,38.0,-84.0,4.0,4,11.0,9.0,Tours,Paddleboarding,Massage,Ziplining,Horsebackriding
1,San Diego,California,United States,33.0,-117.0,4.0,4,16.0,11.0,Beaches,Hiking,Snorkeling,Parasailing,Hot_Air_Balloons
2,Park City,Utah,United States,41.0,-111.0,3.0,4,-3.0,9.0,Tours,Massage,Hiking,Horsebackriding,Ziplining
3,Newcastle Upon Tyne,England,United Kingdom,55.0,-2.0,4.0,4,10.0,10.0,Ziplining,Beaches,Massage,Tours,Horsebackriding
4,Brisbane,Queensland,Australia,-27.0,153.0,4.0,4,21.0,7.0,Ziplining,Massage,Tours,Beaches,Hiking


In [15]:
# Clean final DF Activities to be one of the following list: 
#'beaches','hanggliding','horsebackriding', 'hiking','hot_air_balloons',
#'paddleboarding','parasailing','sailing','snorkeling','ziplining', 
#'Spas','hotsprings','massage','tours'
unique_activities=[]
activities=['act1','act2','act3','act4','act5']
for act in activities:
    unique_activities.extend(step4_df[act].unique())
unique_activities=list(set(unique_activities))
print(unique_activities)


['Horsebackriding', 'Temples', 'Paddleboarding', 'nan', 'Whale Watching', 'Snorkeling', 'Parasailing', 'Tours', 'Wine Tours', 'Hanggliding', 'Beaches', 'Hot_Air_Balloons', 'Spas', 'Hot Springs', 'Sailing', 'Hiking', 'Massage', 'Ziplining', 'Hotsprings', 'Hot Air Balloons', 'Hang Gliding']


In [16]:
#not too bad except we also captured temples, hot air balloons formatted two ways, whale watching, wine tours and tours, 
#hot springs two ways, hang gliding two ways, null values. Need to fix the string values from the yelp activities to the 
#other string format & check with group on keeping whale watching and temples. change wine tours to just tours. Do we also want 
#to change the temp from C to F?

In [20]:
print((step4_df['month']!=4).sum()) 

#In the SQL tables below I'm importing the step4_df index to serve as the locationID # since each city is unique in the rows.
#But then I was thinking, they'd have to appear more than once for the different months, but it looks like there's only april?

#So basically, need to redo the weather merge, then add a new locationID column to the final stage DF to use in all the others
#OR collapse the locations into a single row with lists for months, avg_temp and sun_hours, use the index as the locIDs
#and I can extend out the lists into appropriate columns at the individual table step

0


## Separate Out into Tables for SQL Import

### List of all SQL Tables and Columns

#### Locations
    LocationID(pk), City, Locality, Country, Latitude, Longitude

In [ ]:
#break out locations and assign the step4_df index to be our locationID
locations_df=step4_df[['city','state/province','country','latitude','longitude']]
locations_df.insert(0,'LocationID',step4_df.index)
locations_df.rename(columns={'city':'City','state/province':'Locality','country':'Country',
                             'latitude':'Latitude','longitude':'Longitude'},inplace=True)
locations_df.head()

#### LocationsCharacteristics
    LocationID(pk), LodgingScore, TransportationScore, RestaurantScore

In [ ]:
#locationCharacteristics_df
#were we keeping this?

#### Temperature
    LocationID(pk), January, February, March, April, May, June, July, August, September, October, November, December

In [ ]:
#Split out temperatures, use index for locID, extend lists to columns, rename columns
temperature_df=step4_df[['month','avg_temp']]
temperature_df.insert(0,'LocationID',step4_df.index)
temperature_df.head()


#### SunHours
    LocationID(pk), January, February, March, April, May, June, July, August, September, October, November, December

In [ ]:
sunHours_df=


#### Activities
    LocationID(pk),  Activity1, Activity2, Activity3, Activity4, Activity5

In [ ]:
activities_df=step4_df[['act1','act2','act3','act4','act5']]
activities_df.insert(0,'LocationID',step4_df.index)
activities_df.rename(columns={'act1':'Activity1','act2':'Activity2','act3':'Activity3','act4':'Activity4',
                              'act5':'Activity5'},inplace=True)
activities_df.head()

#### ActivityList
    ActivityID(pk), Name, Image, Attribution, Link

In [ ]:
# add ActivityID
#activityList_df

## Export All Data Into SQL Tables

In [ ]:
# Create outpaths based on ERD
locations_outpath = "../Resources/SQL_tables/locations.csv"
locationCharacteristics_outpath = "../Resources/SQL_tables/locationCharacteristics.csv"
countryFlags_outpath = "../Resources/SQL_tables/countryFlags.csv"
temperature_outpath = "../Resources/SQL_tables/temperature.csv"
sunHours_outpath = "../Resources/SQL_tables/sunHours.csv"
activities_outpath = "../Resources/SQL_tables/activities.csv"
activityList_outpath = "../Resources/SQL_tables/activityList.csv"

In [ ]:
# Export all dataframes
locations_df.to_csv(locations_outpath)
locationCharacteristics_df.to_csv(locationCharacteristics_outpath)
countryFlags_df.to_csv(countryFlags_outpath)
temperature_df.to_csv(temperature_outpath)
sunHours_df.to_csv(sunHours_outpath)
activities_df.to_csv(activities_outpath)
activityList_df.to_csv(activityList_outpath)